In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
games = pd.read_csv(r"../Datasets/games.csv")
stats = pd.read_csv(r"../datasets/International_Chess_Stats.csv")
players = pd.read_csv(r"../datasets/Complete_Players_Database.csv",low_memory=False)

In [3]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20058 entries, 0 to 20057
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              20058 non-null  object 
 1   rated           20058 non-null  bool   
 2   created_at      20058 non-null  float64
 3   last_move_at    20058 non-null  float64
 4   turns           20058 non-null  int64  
 5   victory_status  20058 non-null  object 
 6   winner          20058 non-null  object 
 7   increment_code  20058 non-null  object 
 8   white_id        20058 non-null  object 
 9   white_rating    20058 non-null  int64  
 10  black_id        20058 non-null  object 
 11  black_rating    20058 non-null  int64  
 12  moves           20058 non-null  object 
 13  opening_eco     20058 non-null  object 
 14  opening_name    20058 non-null  object 
 15  opening_ply     20058 non-null  int64  
dtypes: bool(1), float64(2), int64(4), object(9)
memory usage: 2.3+ MB


In [4]:
#splitting out the first ten moves, to see if there's a pattern there, or useful categories
games['opening'] = games['moves'].apply(lambda x : x.split(' ',20))
games['opening'].apply(lambda x : x.pop())
games['opening'] = games['opening'].apply(lambda x : ' '.join(x))

In [5]:
games['increment_code'].unique()

array(['15+2', '5+10', '20+0', '30+3', '10+0', '15+30', '15+0', '20+60',
       '5+40', '8+0', '15+15', '15+16', '11+0', '30+60', '15+5', '10+10',
       '15+3', '18+0', '30+30', '20+25', '5+5', '30+0', '40+40', '16+0',
       '6+10', '8+10', '8+12', '7+12', '15+10', '20+5', '20+10', '45+0',
       '5+8', '30+2', '25+0', '10+2', '7+2', '25+25', '7+8', '6+6',
       '10+4', '12+0', '15+1', '2+10', '9+0', '10+9', '6+7', '30+10',
       '30+15', '10+3', '45+30', '9+2', '7+3', '6+5', '25+5', '5+7',
       '8+2', '20+8', '30+5', '25+15', '15+7', '8+8', '5+9', '16+7',
       '10+7', '7+5', '40+20', '10+5', '11+5', '10+15', '180+0', '17+5',
       '8+5', '3+8', '60+120', '90+120', '15+8', '9+1', '9+3', '2+15',
       '10+1', '5+12', '60+10', '5+60', '8+15', '30+25', '12+1', '14+0',
       '60+20', '10+8', '20+3', '14+12', '7+7', '60+0', '45+45', '4+15',
       '5+6', '180+180', '8+3', '4+6', '10+25', '3+10', '7+4', '6+3',
       '5+15', '45+15', '15+20', '6+8', '6+9', '14+8', '6+4', '15+4',
 

In [6]:
len(games.loc[games['increment_code'].str.startswith('0')]) #check num. of games with unusual time control

45

In [7]:
games['t'] = games['increment_code'].apply(lambda x : x.split('+')[0]).astype(int) #cast time w/o increment as int

In [8]:
games['t'][0]

15

In [9]:
def time_control_label(d):
    if d['t'] < 1:
        return 'Atomic'
    elif d['t'] < 3:
        return 'Bullet'
    elif d['t'] < 10:
        return 'Blitz'
    elif d['t'] >= 15:
        return 'Rapid'
    else:
         return 'Classical'

In [10]:
games['time_ctrl'] = games.apply(lambda x : time_control_label(x), axis=1) #create time controls field

In [11]:
games['time_ctrl'][0]

'Rapid'

In [12]:
del games['t']

In [13]:
# Convert 'created_at' to datetime and save it into a new column: 
games['created_at'] = pd.to_datetime(games['created_at']/1000, unit='s', origin='unix')

# Convert 'last_move_at_modified' to datetime and save it into a new column:
games['last_move_at'] = pd.to_datetime(games['last_move_at']/1000, unit='s', origin='unix')

# Create a new column calculating the time difference between the 2 previous columns:
games['duration'] = pd.to_datetime(games['last_move_at']) - pd.to_datetime(games['created_at'])

games.head(1)

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply,opening,time_ctrl,duration
0,TZJHLljE,False,2017-08-31 20:06:40,2017-08-31 20:06:40,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5,Rapid,0 days


In [14]:
del games['last_move_at']

In [15]:
def rating_dif(game):
    dif = game['white_rating'] - game['black_rating']
    return dif

In [16]:
games['rating_dif'] = games.apply(lambda x : rating_dif(x), axis=1)

In [17]:
games.head(1)

,id,rated,created_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply,opening,time_ctrl,duration,rating_dif
0,TZJHLljE,False,2017-08-31 20:06:40,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5,Rapid,0 days,309


In [18]:
players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175358 entries, 0 to 175357
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Country Rank  175358 non-null  int64 
 1   Name          175358 non-null  object
 2   Title         175358 non-null  object
 3   Country       175358 non-null  object
 4   FIDE          175358 non-null  object
 5   Age           175358 non-null  int64 
 6    K-factor     175358 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 9.4+ MB


In [19]:
print(players.columns)

Index(['Country Rank', 'Name', 'Title', 'Country', 'FIDE', 'Age', ' K-factor'], dtype='object')


In [20]:
print(len(players))
print(players.describe())

175358
        Country Rank            Age       K-factor
count  175358.000000  175358.000000  175358.000000
mean     3221.141704      36.374463      29.539713
std      3779.534044      20.448013      10.328708
min         1.000000       6.000000      10.000000
25%       483.000000      18.000000      20.000000
50%      1653.000000      31.000000      20.000000
75%      4599.000000      53.000000      40.000000
max     16673.000000     970.000000      40.000000


In [21]:
players['Title'].unique()

array(['FM', 'unranked/unrated', 'GM', 'IM', 'WCM', 'WFM', 'WIM', 'WGM',
       'CM', 'WH'], dtype=object)

In [22]:
players['Name'].nunique() #more rows than names, about 1300

174070

In [23]:
players.drop_duplicates(inplace=True)
print(len(players)) #shows no change, no duplicates, just similar names
print(players.describe())

175358
        Country Rank            Age       K-factor
count  175358.000000  175358.000000  175358.000000
mean     3221.141704      36.374463      29.539713
std      3779.534044      20.448013      10.328708
min         1.000000       6.000000      10.000000
25%       483.000000      18.000000      20.000000
50%      1653.000000      31.000000      20.000000
75%      4599.000000      53.000000      40.000000
max     16673.000000     970.000000      40.000000


In [24]:
stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190 entries, 0 to 189
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    190 non-null    int64  
 1   #             190 non-null    int64  
 2   Country       188 non-null    object 
 3   Flag          0 non-null      float64
 4   Num Players   190 non-null    int64  
 5   Women         190 non-null    int64  
 6   % of Women    190 non-null    float64
 7   FIDE Average  190 non-null    int64  
 8   GMs           190 non-null    int64  
 9   IMs           190 non-null    int64  
 10  FMs           190 non-null    int64  
 11  WGMs          190 non-null    int64  
 12  WIMs          190 non-null    int64  
 13  WFMs          190 non-null    int64  
 14  Age Avg       190 non-null    int64  
dtypes: float64(2), int64(12), object(1)
memory usage: 22.4+ KB


In [25]:
# Drop 'Flag':
stats = stats.drop('Flag', axis=1)
# Drop nulls
stats.dropna(inplace=True)

In [26]:
stats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 188 entries, 0 to 189
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    188 non-null    int64  
 1   #             188 non-null    int64  
 2   Country       188 non-null    object 
 3   Num Players   188 non-null    int64  
 4   Women         188 non-null    int64  
 5   % of Women    188 non-null    float64
 6   FIDE Average  188 non-null    int64  
 7   GMs           188 non-null    int64  
 8   IMs           188 non-null    int64  
 9   FMs           188 non-null    int64  
 10  WGMs          188 non-null    int64  
 11  WIMs          188 non-null    int64  
 12  WFMs          188 non-null    int64  
 13  Age Avg       188 non-null    int64  
dtypes: float64(1), int64(12), object(1)
memory usage: 22.0+ KB


In [27]:
# No % can be larger than 100%:

stats[stats['% of Women'] > 100]

,Unnamed: 0,#,Country,Num Players,Women,% of Women,FIDE Average,GMs,IMs,FMs,WGMs,WIMs,WFMs,Age Avg


In [28]:
stats.columns

Index(['Unnamed: 0', '#', 'Country', 'Num Players', 'Women', '% of Women',
       'FIDE Average', 'GMs', 'IMs', 'FMs', 'WGMs', 'WIMs', 'WFMs', 'Age Avg'],
      dtype='object')

In [29]:
print(stats['Unnamed: 0'].head())
stats['Unnamed: 0'].nunique()

0    0
1    1
2    2
3    3
4    4
Name: Unnamed: 0, dtype: int64


188

In [30]:
print(min(stats['Unnamed: 0']))
print(max(stats['Unnamed: 0']))#'Unnamed: 0' is probably an index column and can be dropped

0
189


In [31]:
del stats['Unnamed: 0']

In [32]:
print(stats['#'].head())
print(min(stats['#']))
print(max(stats['#']))# '#' is probably an index column and can be dropped
stats['#'].nunique()

0    1
1    2
2    3
3    4
4    5
Name: #, dtype: int64
1
190


188

In [33]:
del stats['#']

In [34]:
stats.columns

Index(['Country', 'Num Players', 'Women', '% of Women', 'FIDE Average', 'GMs',
       'IMs', 'FMs', 'WGMs', 'WIMs', 'WFMs', 'Age Avg'],
      dtype='object')

In [35]:
players.columns

Index(['Country Rank', 'Name', 'Title', 'Country', 'FIDE', 'Age', ' K-factor'], dtype='object')

In [36]:
games.columns

Index(['id', 'rated', 'created_at', 'turns', 'victory_status', 'winner',
       'increment_code', 'white_id', 'white_rating', 'black_id',
       'black_rating', 'moves', 'opening_eco', 'opening_name', 'opening_ply',
       'opening', 'time_ctrl', 'duration', 'rating_dif'],
      dtype='object')

In [37]:
games = games[['id','rated','created_at','duration','turns','victory_status','winner','time_ctrl','increment_code','white_id','white_rating','black_id','black_rating','rating_dif','moves','opening_eco','opening_name','opening_ply','opening']]

In [38]:
print(games.columns)
games.head()

Index(['id', 'rated', 'created_at', 'duration', 'turns', 'victory_status',
       'winner', 'time_ctrl', 'increment_code', 'white_id', 'white_rating',
       'black_id', 'black_rating', 'rating_dif', 'moves', 'opening_eco',
       'opening_name', 'opening_ply', 'opening'],
      dtype='object')


,id,rated,created_at,duration,turns,victory_status,winner,time_ctrl,increment_code,white_id,white_rating,black_id,black_rating,rating_dif,moves,opening_eco,opening_name,opening_ply,opening
0,TZJHLljE,False,2017-08-31 20:06:40,0 days,13,outoftime,white,Rapid,15+2,bourgris,1500,a-00,1191,309,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5
1,l1NXvwaE,True,2017-08-30 21:53:20,0 days,16,resign,black,Blitz,5+10,a-00,1322,skinnerua,1261,61,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...
2,mIICvQHh,True,2017-08-30 21:53:20,0 days,61,mate,white,Blitz,5+10,ischia,1496,a-00,1500,-4,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...
3,kWKvrqYL,True,2017-08-30 16:20:00,0 days,61,mate,white,Rapid,20+0,daniamurashov,1439,adivanov2009,1454,-15,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...
4,9tXo1AUZ,True,2017-08-29 18:06:40,0 days,95,mate,white,Rapid,30+3,nik221107,1523,adivanov2009,1469,54,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...


In [39]:
stats.to_csv('../datasets/stats_clean.csv')
players.to_csv('../datasets/players_clean.csv')
games.to_csv('../datasets/games_clean.csv')